In [10]:
import pandas as pd
from chanlun import cl
from chanlun import kcharts

### 避免在 Notebook 中不显示，手动加载下依赖的js
###　https://pyecharts.org/#/zh-cn/notebook
from pyecharts.charts import Kline, Line, Bar, Grid, Scatter
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB
Kline().load_javascript()
Line().load_javascript()
Bar().load_javascript()
Grid().load_javascript()
Scatter().load_javascript()

In [11]:
# 获取 行情K线数据
code = 'SH.688122'
frequency = '30m'
klines = pd.read_csv('./data/688122.csv')
klines


,code,date,open,close,high,low,volume
0,SH.688122,2021-09-27 10:00:00,75.67,73.99,76.80,73.40,1142100.0
1,SH.688122,2021-09-27 10:30:00,73.80,72.18,73.98,71.60,2450700.0
2,SH.688122,2021-09-27 11:00:00,72.19,71.33,72.71,70.83,1041900.0
3,SH.688122,2021-09-27 11:30:00,71.17,69.81,71.67,69.00,1369800.0
4,SH.688122,2021-09-27 13:30:00,70.10,73.11,73.11,69.81,1074100.0
5,SH.688122,2021-09-27 14:00:00,73.01,73.14,73.35,72.80,617200.0
6,SH.688122,2021-09-27 14:30:00,73.14,73.17,73.49,73.00,351800.0
7,SH.688122,2021-09-27 15:00:00,73.17,72.75,73.44,72.75,712500.0
8,SH.688122,2021-09-28 10:00:00,72.75,74.20,74.29,72.05,863600.0
9,SH.688122,2021-09-28 10:30:00,74.20,74.33,75.10,73.59,595700.0


In [12]:
# 依据 K 线数据，计算缠论数据
cl_data = cl.CL(code, frequency).process_klines(klines)
chart = kcharts.render_charts('%s - %s' % (code, frequency), cl_data)

In [13]:
# 展示图标
chart

In [19]:
# 获取最后一笔
bi = cl_data.bis[-1]
print('Last Bi : ', bi)
# 获取最后一个中枢
zs = cl_data.zss[-1]
print('Last Zs : ', zs)

Last Bi :  index: 38 type: up FX: (346 - 366) high: 85.67 low: 73.7 done: True
Last Zs :  index: 19 level: 0 FX: (321-366) type: zd zg: 78.87 zd: 76.0 gg: 85.67 dd: 73.7 done: False


In [24]:
# 判断笔是否背驰，是否有买卖点
bi = cl_data.bis[-25]
print('BI MMDS : ', bi.bi_mmds())
print('BI BC : ', [_bc.type for _bc in bi.bcs])
print('BI 笔背驰：', bi.bc_exists(['bi']))
print('BI 盘整背驰：', bi.bc_exists(['pz']))
print('BI 趋势背驰：', bi.bc_exists(['qs']))

BI MMDS :  ['1sell']
BI BC :  ['bi', 'pz', 'qs']
BI 笔背驰： False
BI 盘整背驰： True
BI 趋势背驰： True


In [26]:
# 买卖点信息
mmd = bi.mmds[0]
print(mmd)
print('买卖点对应的中枢：', mmd.zs)

MMD: 1sell ZS: index: 5 level: 2 FX: (114-283) type: zd zg: 93.48 zd: 85.4 gg: 99.92 dd: 77.38 done: True
买卖点对应的中枢： index: 5 level: 2 FX: (114-283) type: zd zg: 93.48 zd: 85.4 gg: 99.92 dd: 77.38 done: True


## Html 页面展示

### 服务端代码
    return HttpResponse(kcharts.render_charts('标题', cl_datas))

### 页面 Ajax 接收并展示
    var chart = echarts.init(document.getElementById('chart_id'), 'white', {renderer: 'canvas'});
    $.ajax({
        type: "GET",
        url: "/后端请求地址",
        success: function (result) {
            var res = (new Function("return " + result))();
            chart.setOption(res);
        }
    });